<a href="https://colab.research.google.com/github/evndttr/htr_scripts/blob/main/Passim_with_ALTO_xml_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!java --version

openjdk 11.0.28 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [3]:
!pip install git+https://github.com/dasmiq/passim.git

  Cloning https://github.com/dasmiq/passim.git to /tmp/pip-req-build-lxr41avr
  Running command git clone --filter=blob:none --quiet https://github.com/dasmiq/passim.git /tmp/pip-req-build-lxr41avr
  Resolved https://github.com/dasmiq/passim.git to commit e4eea1c89c4182094d119047c85032cd9163ea17
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.4 MB/s eta 0:00:00
  Created wheel for passim: filename=passim-2.0.0-py3-none-any.whl size=15556 sha256=8290a6b426d14585121289a6720decafcb6a0486ad5b7497bbd70635e2b77ef1
  Stored in directory: /tmp/pip-ephem-wheel-cache-xu7axh9e/wheels/b0/51/a2/50fb4f12befcc05270d5ad947734813b1e14df43e9f21862ad
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=1dcbbdf1f9fe607ae12f0d050ea21edad76648201ed39e2928507bdbb1ec726a
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa74a8e16e958

In [4]:
!seriatim --help

https://repos.spark-packages.org/ added as a remote repository with the name: repo-1
:: loading settings :: url = jar:file:/usr/local/lib/python3.12/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de1be91f-b371-4008-8de6-1644c54b5b8d;1.0
	confs: [default]
	found graphframes#graphframes;0.8.4-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.4-spark3.5-s_2.12/graphframes-0.8.4-spark3.5-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.4-spark3.5-s_2.12!graphframes.jar (62ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (128ms)
:: 

In [5]:
import json
from urllib.request import urlopen
import sys
import passim

In [7]:
import xml.etree.ElementTree as ET

# Path to your uploaded file
xml_path = "/content/1.xml"

# Parse the XML
tree = ET.parse(xml_path)
root = tree.getroot()

# Namespace (ALTO v4)
ns = {'a': 'http://www.loc.gov/standards/alto/ns-v4#'}

# Collect each line's words separately
lines = []
for textline in root.findall('.//a:TextLine', ns):
    words = [el.attrib['CONTENT'] for el in textline.findall('.//a:String', ns)]
    line_text = " ".join(words)
    lines.append(line_text)

# Join all lines into one text block, keeping newlines
raw_ocr = "\n".join(lines)

# Optional: print a preview
print(raw_ocr[:500] + "...")
print(f"\nTotal characters: {len(raw_ocr)}")


L
Pres Que missire Chaxles
de mouy Cheuallier seigneur de la milleraye et uisadumral
de france ent prins Les sermens et fanct iurez Les
Cappitaine maistres et conpaignons desditz Namres
de bien et Loyaulment soy porter au seruice du l
soulz La charge dudit Cartier
Dartimes du haure et port de zamet male anecques
Iesdy deuy Namres duport demiron soixantr touueaulx
t
chameun esquippez Les deux de saixante ung homme Le
uigntuesme four dapuois ondit ꝑo Illis Ciiq ceus teante...

Total characters: 475


In [8]:
print(raw_ocr)

L
Pres Que missire Chaxles
de mouy Cheuallier seigneur de la milleraye et uisadumral
de france ent prins Les sermens et fanct iurez Les
Cappitaine maistres et conpaignons desditz Namres
de bien et Loyaulment soy porter au seruice du l
soulz La charge dudit Cartier
Dartimes du haure et port de zamet male anecques
Iesdy deuy Namres duport demiron soixantr touueaulx
t
chameun esquippez Les deux de saixante ung homme Le
uigntuesme four dapuois ondit ꝑo Illis Ciiq ceus teante


In [10]:

# --- Load text from files instead of inline strings ---
with open("/content/relation.txt", encoding="utf-8", newline=None) as f:
    relation = f.read()

docs = [
    {"id": "relation", "group": "authority", "text": relation}
]

In [12]:
# Add the ocr from the xml into the documents
text = raw_ocr

docs.append({
    'id': '/ocr_msmoreau/f1',
    'group': '/ocr_msmoreau',
    'text': text
})

In [13]:
with open('in.json', 'w') as f:
  for d in docs:
    print(json.dumps(d), file=f)

In [14]:
# First deletes old output
!rm -r out_cluster
!SPARK_SUBMIT_ARGS="--driver-memory 8G --executor-memory 8G" seriatim in.json out_cluster >& out_cluster.err

rm: cannot remove 'out_cluster': No such file or directory


In [15]:
!rm -r out_cluster
!SPARK_SUBMIT_ARGS="--driver-memory 8G --executor-memory 8G" \
seriatim in.json out_cluster \
-n 15 \
-m 2 \
>& out_cluster.err

In [16]:
import glob, itertools

# Read one JSON record per line
def read_jsonl_file(f):
  res = []
  for line in f:
    res.append(json.loads(line))
  return res

def read_jsonl(d):
  return list(itertools.chain.from_iterable([read_jsonl_file(open(f)) for f in glob.glob(d + '/*.json')]))

In [17]:
read_jsonl('out_cluster/out.json')

[{'uid': -1381474889189377111,
  'cluster': 8589934592,
  'begin': 2,
  'end': 475,
  'boiler': False,
  'src': [{'uid': -4245069274104881227, 'begin': 29, 'end': 518}],
  'size': 2,
  'pboiler': 0.0,
  'group': '/ocr_msmoreau',
  'id': '/ocr_msmoreau/f1',
  'text': 'Pres Que missire Chaxles\nde mouy Cheuallier seigneur de la milleraye et uisadumral\nde france ent prins Les sermens et fanct iurez Les\nCappitaine maistres et conpaignons desditz Namres\nde bien et Loyaulment soy porter au seruice du l\nsoulz La charge dudit Cartier\nDartimes du haure et port de zamet male anecques\nIesdy deuy Namres duport demiron soixantr touueaulx\nt\nchameun esquippez Les deux de saixante ung homme Le\nuigntuesme four dapuois ondit ꝑo Illis Ciiq ceus teante'},
 {'uid': -4245069274104881227,
  'cluster': 8589934592,
  'begin': 29,
  'end': 518,
  'boiler': False,
  'src': [],
  'size': 2,
  'pboiler': 0.0,
  'group': 'authority',
  'id': 'relation',
  'text': "pres que Missire Charles de Mouy, chevalli